In [1]:
pip install mysql-connector-python

   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 15.1/15.1 MB 499.0 kB/s eta 0:00:0000:0100:01

[notice] A new release of pip is available: 25.0 -> 25.0.1
[notice] To update, run: pip install --upgrade pip
Note: you may need to restart the kernel to use updated packages.


In [7]:
import mysql.connector
import pandas as pd

# Establish connection
conn = mysql.connector.connect(
    host='localhost',
    user='root',
    password='123456789',  # if you have password, insert here
    database='StudentInformationSystem',
    port=3306
)

# Cursor to execute SQL commands
cursor = conn.cursor()

# Step 1: Get all table names from your schema
cursor.execute("SHOW TABLES")
tables = cursor.fetchall()

# Step 2: Fetch data from each table
all_dataframes = {}

for (table_name,) in tables:
    query = f"SELECT * FROM {table_name}"
    df = pd.read_sql(query, con=conn)
    all_dataframes[table_name] = df
    print(f"\nTable: {table_name}")
    print(df.head())  # Display first 5 rows for preview

# Step 3: Close connections
cursor.close()
conn.close()


Table: Academic_Qualification
Empty DataFrame
Columns: [academic_id, student_id, exam_type, board_or_university, school_or_college, passing_year, percentage_or_cgpa]
Index: []

Table: Address
   address_id flat_no_or_plot_no          landmark    area    city    state  \
0           1                101  Near City Center  Area-1  City-1  State-1   
1           2                102  Near City Center  Area-2  City-2  State-2   
2           3                103  Near City Center  Area-3  City-3  State-3   
3           4                104  Near City Center  Area-4  City-4  State-4   
4           5                105  Near City Center  Area-5  City-5  State-5   

  country pincode  
0   India   50001  
1   India   50002  
2   India   50003  
3   India   50004  
4   India   50005  

Table: Attendance
   attendance_id  course_id  student_id attendance_date   status  \
0              1          1        7001      2023-08-01  Present   
1              2          2        7002      2023-08-01  

/var/folders/hy/d5zmwm0x24ncdhw2w7md39wh0000gn/T/ipykernel_10161/3732461805.py:25: UserWarning: pandas only supports SQLAlchemy connectable (engine/connection) or database string URI or sqlite3 DBAPI2 connection. Other DBAPI2 objects are not tested. Please consider using SQLAlchemy.
  df = pd.read_sql(query, con=conn)
/var/folders/hy/d5zmwm0x24ncdhw2w7md39wh0000gn/T/ipykernel_10161/3732461805.py:25: UserWarning: pandas only supports SQLAlchemy connectable (engine/connection) or database string URI or sqlite3 DBAPI2 connection. Other DBAPI2 objects are not tested. Please consider using SQLAlchemy.
  df = pd.read_sql(query, con=conn)
/var/folders/hy/d5zmwm0x24ncdhw2w7md39wh0000gn/T/ipykernel_10161/3732461805.py:25: UserWarning: pandas only supports SQLAlchemy connectable (engine/connection) or database string URI or sqlite3 DBAPI2 connection. Other DBAPI2 objects are not tested. Please consider using SQLAlchemy.
  df = pd.read_sql(query, con=conn)
/var/folders/hy/d5zmwm0x24ncdhw2w7md39wh

In [10]:
for table, df in all_dataframes.items():
    df.to_csv(f"{table}.csv", index=False)
    print(f"Saved {table}.csv")

Saved Academic_Qualification.csv
Saved Address.csv
Saved Attendance.csv
Saved Class_10.csv
Saved Class_12.csv
Saved Course.csv
Saved Course_Attendance.csv
Saved Degree.csv
Saved Department.csv
Saved Employment_Type.csv
Saved Enrollment.csv
Saved Faculty.csv
Saved Family_Income.csv
Saved Father.csv
Saved Grade.csv
Saved Guardian_Information.csv
Saved Mother.csv
Saved Name.csv
Saved Parent_Information.csv
Saved Personal_Details.csv
Saved Program.csv
Saved Semester_Wise_Marks.csv
Saved Student.csv
Saved Student_ID_Table.csv
Saved Term.csv
Saved Work_Experience.csv


In [18]:
pip install streamlit pandas plotly

   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 14.8/14.8 MB 8.5 MB/s eta 0:00:00a 0:00:01

[notice] A new release of pip is available: 25.0 -> 25.0.1
[notice] To update, run: pip install --upgrade pip
Note: you may need to restart the kernel to use updated packages.


In [23]:
streamlit run app.py

SyntaxError: invalid syntax (3737097518.py, line 1)

In [24]:
import streamlit as st
import pandas as pd
import plotly.express as px

# Title and Navigation
st.set_page_config(page_title="Student Support System", layout="wide")
st.title("📚 Student Support System")

# Sidebar Navigation
st.sidebar.title("Navigation")
option = st.sidebar.radio("Go to", ["Student Info", "Grades", "Attendance"])

# Load Data
@st.cache_data
def load_data():
    grades = pd.read_csv("Grade.csv")
    attendance = pd.read_csv("Attendance.csv")
    student_info = pd.read_csv("Student_ID_Table.csv")
    return grades, attendance, student_info

grades_df, attendance_df, student_info_df = load_data()

# Tab: Student Info
if option == "Student Info":
    st.header("👩‍🎓 Student Overview")
    st.dataframe(student_info_df)

# Tab: Grades
elif option == "Grades":
    st.header("📊 Grades Report")

    st.subheader("Total Marks per Student")
    fig1 = px.bar(grades_df, x='student_id', y='total_marks', color='status',
                  title="Total Marks and Pass/Fail Status")
    st.plotly_chart(fig1, use_container_width=True)

    st.subheader("CIA Performance")
    cia_data = grades_df[['student_id', 'cia1', 'cia2', 'cia3', 'cia4']].melt(id_vars='student_id',
                var_name='CIA Exam', value_name='Marks')
    fig2 = px.box(cia_data, x="CIA Exam", y="Marks", color="CIA Exam", title="CIA Marks Distribution")
    st.plotly_chart(fig2, use_container_width=True)

# Tab: Attendance
elif option == "Attendance":
    st.header("📅 Attendance Tracker")

    st.subheader("Attendance Summary")
    att_summary = attendance_df['status'].value_counts().reset_index()
    att_summary.columns = ['Status', 'Count']
    fig3 = px.pie(att_summary, values='Count', names='Status', title='Attendance Overview')
    st.plotly_chart(fig3, use_container_width=True)

    st.subheader("Detailed Attendance Data")
    st.dataframe(attendance_df)

2025-03-24 22:42:37.777 Thread 'MainThread': missing ScriptRunContext! This warning can be ignored when running in bare mode.
2025-03-24 22:42:37.807 Thread 'MainThread': missing ScriptRunContext! This warning can be ignored when running in bare mode.
2025-03-24 22:42:37.808 Thread 'MainThread': missing ScriptRunContext! This warning can be ignored when running in bare mode.
2025-03-24 22:42:37.809 Thread 'MainThread': missing ScriptRunContext! This warning can be ignored when running in bare mode.
2025-03-24 22:42:37.809 Thread 'MainThread': missing ScriptRunContext! This warning can be ignored when running in bare mode.
2025-03-24 22:42:37.810 Thread 'MainThread': missing ScriptRunContext! This warning can be ignored when running in bare mode.
2025-03-24 22:42:37.810 Thread 'MainThread': missing ScriptRunContext! This warning can be ignored when running in bare mode.
2025-03-24 22:42:37.811 Thread 'MainThread': missing ScriptRunContext! This warning can be ignored when running in bar

In [25]:
import streamlit as st
import pandas as pd
import os

# Set the page config
st.set_page_config(page_title="Student Support System", layout="wide")

# Title
st.title("📚 Student Support System Dashboard")

# Load data files
attendance_path = "Attendance.csv"
grade_path = "Grade.csv"

if os.path.exists(attendance_path) and os.path.exists(grade_path):
    attendance_df = pd.read_csv(attendance_path)
    grade_df = pd.read_csv(grade_path)

    # Sidebar options
    option = st.sidebar.radio("Select View", ["📊 Attendance Summary", "🎓 Grade Performance"])

    if option == "📊 Attendance Summary":
        st.header("📊 Attendance Status of Students")
        status_count = attendance_df['status'].value_counts()
        st.bar_chart(status_count)

        st.subheader("🧾 Full Attendance Data")
        st.dataframe(attendance_df)

    elif option == "🎓 Grade Performance":
        st.header("🎓 Total Marks per Student")
        fig_data = grade_df[['student_id', 'total_marks', 'status']]
        st.bar_chart(
            data=fig_data,
            x='student_id',
            y='total_marks',
        )

        # Pass/Fail breakdown
        st.subheader("✅ Pass / ❌ Fail Breakdown")
        grade_status = grade_df['status'].value_counts()
        st.write(grade_status)

        st.subheader("🧾 Full Grade Data")
        st.dataframe(grade_df)

else:
    st.error("📁 Required CSV files not found in current directory!")

2025-03-24 22:48:17.644 Thread 'MainThread': missing ScriptRunContext! This warning can be ignored when running in bare mode.
2025-03-24 22:48:17.646 Thread 'MainThread': missing ScriptRunContext! This warning can be ignored when running in bare mode.
2025-03-24 22:48:17.646 Thread 'MainThread': missing ScriptRunContext! This warning can be ignored when running in bare mode.
2025-03-24 22:48:17.667 Thread 'MainThread': missing ScriptRunContext! This warning can be ignored when running in bare mode.
2025-03-24 22:48:17.667 Thread 'MainThread': missing ScriptRunContext! This warning can be ignored when running in bare mode.
2025-03-24 22:48:17.668 Thread 'MainThread': missing ScriptRunContext! This warning can be ignored when running in bare mode.
2025-03-24 22:48:17.668 Thread 'MainThread': missing ScriptRunContext! This warning can be ignored when running in bare mode.
2025-03-24 22:48:17.669 Thread 'MainThread': missing ScriptRunContext! This warning can be ignored when running in bar